# Getting used to PythonQT

date = 30 September, 2020 <br>

history <br>
2020-30-09 Get first PythonQt window <br>
2020-30-09 Combination PythonQT + VTK content <br>
2020-04-10 Change Directory and Time Step and toogle background button <br>

In [ ]:
import sys
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QToolBar, QAction, QStatusBar, QCheckBox
from PyQt5.QtCore import Qt, QSize
from PyQt5.QtGui import QIcon

### First example

In [ ]:
class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)
        
        self.setWindowTitle("My Awesome App")
        
        label = QLabel("THIS IS AWESOME!!!")
        label.setAlignment(Qt.AlignCenter)
        
        self.setCentralWidget(label)
        
        toolbar = QToolBar("My main toolbar")
        toolbar.setIconSize(QSize(16,16))
        self.addToolBar(toolbar)
        
        button_action = QAction(QIcon("bug.png"), "Your button", self)
        button_action.setStatusTip("This is your button")
        button_action.triggered.connect(self.onMyToolBarButtonClick)
        button_action.setCheckable(True)
        toolbar.addAction(button_action)
        
        toolbar.addSeparator()
        
        button_action2 = QAction(QIcon("bug.png"), "Your button2", self)
        button_action2.setStatusTip("This is your button2")
        button_action2.triggered.connect(self.onMyToolBarButtonClick)
        button_action2.setCheckable(True)
        toolbar.addAction(button_action2)
        
        toolbar.addWidget(QLabel("Hello"))
        toolbar.addWidget(QCheckBox())
        
        self.setStatusBar(QStatusBar(self))
        
    def onMyToolBarButtonClick(self, s):
        print("click", s)

app = QApplication(sys.argv)
window = MainWindow()
window.show()

app.exec_()

## First Python QT + VTK

In [ ]:
import vtk
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor
from PyQt5.QtWidgets import QFrame, QVBoxLayout

In [ ]:
class MainWindow(QMainWindow):

    def __init__(self, parent = None):
        QMainWindow.__init__(self, parent)
        
        self.setWindowTitle("KidsBrainProject")
        
        # add vtk widget
        self.frame = QFrame()
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.vl = QVBoxLayout()
        self.vl.addWidget(self.vtkWidget)
        
        # create and add renderer
        self.renderer = vtk.vtkRenderer()  
        self.vtkWidget.GetRenderWindow().AddRenderer(self.renderer)
        self.interactor = self.vtkWidget.GetRenderWindow().GetInteractor() 
        
        # Create source
        source = vtk.vtkSphereSource()
        source.SetCenter(0, 0, 0)
        source.SetRadius(5.0)
        # Create a mapper
        mapper = vtk.vtkPolyDataMapper()
        mapper.SetInputConnection(source.GetOutputPort())
        # Create an actor
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        
        self.renderer.AddActor(actor)
        self.renderer.ResetCamera()
        
        self.frame.setLayout(self.vl)
        self.setCentralWidget(self.frame)
        
        self.show()
        self.interactor.Initialize() 
        

# Python QT + VTK + ImageViewer

In [ ]:
class MainWindow(QMainWindow):

    def __init__(self, parent = None):
        QMainWindow.__init__(self, parent)
        
        self.setWindowTitle("KidsBrainProject")
        
        # add vtk widget
        self.frame = QFrame()
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.vl = QVBoxLayout()
        self.vl.addWidget(self.vtkWidget)
                
        # Read data
        pathDicomDir = "../../Data/1/CT"
        reader = vtk.vtkDICOMImageReader()
        reader.SetDirectoryName(pathDicomDir)
        reader.Update()

        # Create image viewer
        self.viewer = vtk.vtkImageViewer2()
        self.viewer.SetInputData(reader.GetOutput())
        self.viewer.SetSlice(100)
                
        self.viewer.SetupInteractor(self.vtkWidget)
        self.viewer.SetRenderWindow(self.vtkWidget.GetRenderWindow())
        self.viewer.Render()
        self.frame.setLayout(self.vl)
        self.setCentralWidget(self.frame)
        
        self.show()
        self.vtkWidget.Initialize()
        
    def keyPressEvent(self, event):
        print(event.key())
        if event.key() == QtCore.Qt.Key_Escape: 
            self.lastWindowClosed()
        

# PythonQT + VTK + ImageViewer + Change Directory/TimeStep

In [ ]:
from PyQt5.QtWidgets import QPushButton, QFileDialog, QSlider

In [ ]:
import os

In [ ]:
path_dir = "../../Data/1/"
time_steps = [x for x in os.listdir(path_dir) if 'CT' in x or 'T1' in x]

In [ ]:
class MainWindow(QMainWindow):

    def __init__(self, parent = None):
        QMainWindow.__init__(self, parent)
        
        init_time_step = 0
              
        # Set up frame
        self.frame = QFrame()
        self.layout = QVBoxLayout()
                              
         # VTK widget
        self.vtkWidget = QVTKRenderWindowInteractor(self.frame)
        self.layout.addWidget(self.vtkWidget)
               
        # Read data
        self.path_dir = "../../Data/1/"
        self.reader = vtk.vtkDICOMImageReader()
        self.changeReader(nr_time_step=0)

        # Create image viewer
        self.viewer = vtk.vtkImageViewer2()
        self.viewer.SetInputData(self.reader.GetOutput())
        self.viewer.SetSlice(100)
        #self.viewer.SetColorWindow(0)
        #self.viewer.SetColorLevel(0)
        self.colorWindow = self.viewer.GetColorWindow()
        self.colorLevel = self.viewer.GetColorLevel()
                
        # Connect VTK + QT
        self.viewer.SetupInteractor(self.vtkWidget)
        self.viewer.SetRenderWindow(self.vtkWidget.GetRenderWindow())
        self.viewer.Render()
        self.frame.setLayout(self.layout)
        self.setCentralWidget(self.frame)
        
        # Toolbar with Slider
        self.toolbar = QToolBar("Time slider") 
        self.addToolBar(self.toolbar)
        self.createSliderToolbar()        
        
        # Menu Bar
        bar = self.menuBar()
        file = bar.addMenu("File")
        
        new_folder = QAction("New Patient",self)
        new_folder.triggered.connect(self.openFolder)
        #new_folder.setShortcut("Ctrl+N")
        file.addAction(new_folder)
        
        quit = QAction("Quit",self) 
        quit.triggered.connect(self.closeWindow)
        file.addAction(quit)
        
        # Show and initialize
        self.show()
        self.vtkWidget.Initialize()
        
    def closeWindow(self, s):
        self.close()
        
    def createSliderToolbar(self, init_value=0):
        self.removeToolBar(self.toolbar)
        self.toolbar = QToolBar("Time slider") 
        
        self.slider = QSlider(Qt.Horizontal)
        self.slider.valueChanged.connect(self.timeStepChange)
        self.slider.setMinimum(0)
        self.slider.setMaximum(len(self.time_steps)-1)
        if init_value > len(self.time_steps)-1:
            init_value = len(self.time_steps)-1
        self.slider.setValue(init_value)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(1)    
        print('Create slider {0}/{1}'.format(init_value, len(self.time_steps)-1))
        self.toolbar.addWidget(self.slider)
        
        self.toolbar.addSeparator()
        
        background_toggle = QCheckBox()
        background_toggle.setText("Background Toggle")
        background_toggle.setChecked(True)
        background_toggle.stateChanged.connect(self.backgroundToogleChange)
        
        self.toolbar.addWidget(background_toggle)
        self.addToolBar(self.toolbar)
        
    def backgroundToogleChange(self, s):
        if s == 0:
            print('no background')
            self.colorWindow = self.viewer.GetColorWindow()
            self.colorLevel = self.viewer.GetColorLevel()
            self.viewer.SetColorLevel(255)
            self.viewer.SetColorWindow(0)
            self.viewer.Render()
        elif s == 2:
            print('background')
            self.viewer.SetColorLevel(self.colorLevel)
            self.viewer.SetColorWindow(self.colorWindow)
            self.viewer.Render()
        
    def openFolder(self, s):
        
        file_dialog = QFileDialog()
        file_dialog.setFileMode(QFileDialog.DirectoryOnly)
        file_dialog.setViewMode(QFileDialog.Detail)
        
        if file_dialog.exec_():
            self.path_dir = file_dialog.selectedFiles()[0]
            print('Open folder', self.path_dir)
            self.changeReader()
            self.createSliderToolbar(init_value=self.slider.value())
            
    def changeReader(self, nr_time_step=None):
        self.time_steps = [x for x in os.listdir(self.path_dir) if 'CT' in x or 'T1' in x]
        if nr_time_step is None:
            nr_time_step = min(len(self.time_steps)-1,self.slider.value())
            self.slider.setValue(nr_time_step)
        self.path_dicom_dir = os.path.join(self.path_dir, self.time_steps[nr_time_step]) 
        self.reader.SetDirectoryName(self.path_dicom_dir)
        self.reader.Update()
        print('Change folder', self.path_dicom_dir)

            
    def timeStepChange(self,s):
        print('Change step {0}/{1}'.format(self.slider.value(),len(self.time_steps)-1))
        self.changeReader()        

In [ ]:
app = QApplication(sys.argv)
window = MainWindow()
window.show()

app.exec_()

In [ ]:
window.viewer.GetColorLevel(), window.viewer.GetColorWindow()